In [ ]:
#  http://webgraphviz.com/

In [121]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns, pydotplus
%matplotlib inline
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [122]:
df = pd.read_csv(r'G:\My Drive\Creative_Meta_Analysis\Data\Clean_Data_Top2_Recode.csv',
                 encoding='latin1', 
                 na_values='', 
                 keep_default_na=False)

In [123]:
df = df[
    (df['Platform'] == 'Desktop') & 
    (df['Region'] == 'NA') & 
    (df['Format'] == 'Video') & 
    (df['Set_Cell'] != 0) &
    (df['Video_Length'] == ':15')
]

In [124]:
df.shape

(21109, 26)

In [125]:
df.tail()

,Country,Region,Quarter_Year,Campaign,Gender,Operating_System,Aided_Awareness,Purchase_Intent,Consideration_Intent,Enjoyment,Interesting,Unique,Involving,Understanding,Relevance,Branding,Brand_Appeal,Distinctiveness,Set_Cell,Creative_Name,Format,Publisher,Platform,Product,Video_Length,Age_Recode
265648,US,NA,Q4 2017,US Video Google Home Max Desktop,0.0,NaN,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,OOGL3291000H_Google_Home-_Biggie_First_Listen_...,Video,YouTube,Desktop,Google Home Max,:15,3.0
265650,US,NA,Q4 2017,US Video Google Home Max Desktop,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,OOGL3291000H_Google_Home-_Biggie_First_Listen_...,Video,YouTube,Desktop,Google Home Max,:15,3.0
265655,US,NA,Q4 2017,US Video Google Home Max Desktop,0.0,NaN,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,OOGL3291000H_Google_Home-_Biggie_First_Listen_...,Video,YouTube,Desktop,Google Home Max,:15,2.0
265661,US,NA,Q4 2017,US Video Google Home Max Desktop,0.0,NaN,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0,OOGL3291000H_Google_Home-_Biggie_First_Listen_...,Video,YouTube,Desktop,Google Home Max,:15,3.0
265664,US,NA,Q4 2017,US Video Google Home Max Desktop,0.0,NaN,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,2.0,OOGL3291000H_Google_Home-_Biggie_First_Listen_...,Video,YouTube,Desktop,Google Home Max,:15,3.0


### Decision Trees

In [126]:
new_df = df[[
    'Consideration_Intent',
    'Enjoyment',
    'Involving',
    'Interesting',
    'Unique',
    'Understanding',
    'Relevance',
    'Branding',
    'Brand_Appeal',
    'Distinctiveness',
    'Age_Recode',
    'Video_Length'
]].dropna()

vid = pd.get_dummies(new_df['Video_Length'], columns=[':06', ':15', ':30'])
final_df = pd.concat([new_df, vid], axis=1)

In [127]:
from sklearn.model_selection import train_test_split

X = new_df.drop(['Video_Length', 'Consideration_Intent'], axis=1)
y = new_df['Consideration_Intent']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [128]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import GridSearchCV

dt_aided = DecisionTreeClassifier()
dt_parameters = [{
        'max_depth': [x for x in range(1, 10)]
}]

dt_grid = GridSearchCV(dt_aided, dt_parameters)

In [115]:
dt_train = dt_grid.fit(X_train, y_train)

In [129]:
pd.DataFrame(dt_train.cv_results_)

c:\users\william.raikes\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
c:\users\william.raikes\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
c:\users\william.raikes\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_tra

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_max_depth,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,0.002337,0.000503,0.654762,0.654765,1,{'max_depth': 1},6,0.651992,0.656151,0.640756,0.661765,0.671579,0.646380,0.000149,4.761893e-04,0.012729,0.006357
1,0.001467,0.002122,0.663866,0.669471,2,{'max_depth': 2},3,0.666667,0.668770,0.640756,0.679622,0.684211,0.660021,0.001039,1.519458e-03,0.017841,0.008017
2,0.001499,0.000000,0.666667,0.677874,3,{'max_depth': 3},1,0.666667,0.678233,0.649160,0.685924,0.684211,0.669465,0.001469,0.000000e+00,0.014302,0.006724
3,0.004085,0.000334,0.664566,0.679625,4,{'max_depth': 4},2,0.666667,0.680336,0.647059,0.688025,0.680000,0.670514,0.002940,4.726067e-04,0.013523,0.007167
4,0.004504,0.000441,0.658964,0.686280,5,{'max_depth': 5},4,0.654088,0.689800,0.642857,0.694328,0.680000,0.674711,0.001924,6.237734e-04,0.015544,0.008386
5,0.004254,0.000531,0.658964,0.691883,6,{'max_depth': 6},4,0.656184,0.696109,0.636555,0.700630,0.684211,0.678909,0.002129,7.513378e-04,0.019545,0.009358
6,0.004874,0.001002,0.652661,0.699585,7,{'max_depth': 7},7,0.654088,0.704522,0.628151,0.704832,0.675789,0.689402,0.003668,1.946680e-07,0.019464,0.007202
7,0.002001,0.000334,0.652661,0.706236,8,{'max_depth': 8},7,0.662474,0.707676,0.621849,0.713235,0.673684,0.697796,0.000746,4.724943e-04,0.022263,0.006385
8,0.001839,0.000668,0.651261,0.710787,9,{'max_depth': 9},9,0.656184,0.710831,0.628151,0.717437,0.669474,0.704092,0.000122,4.722696e-04,0.017218,0.005448


### Visualize Tree

In [132]:
export_graphviz(dt_train.best_estimator_,
                out_file='recode_tree.dot',
                rounded=True,
                feature_names=X_train.columns,
                filled=True,
                class_names=['Not_Considered', 'Considered'])

### Accuracy Score

In [131]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, dt_train.best_estimator_.predict(X_test))

0.6289791437980241